# Bosch Kaggle Script
## To Do:
* Objectify
* LabelEncode Data?
* Decision Tree


In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import subprocess
import time
import sys

In [6]:
# Data handling stuff
in_csv = 'data/train_categorical.csv'
# Chunksize seems to work for 32GB of RAM
chunksize = 10000


# nlines = subprocess.check_output('wc -l %s' % in_csv, shell=True)
# nlines = int(nlines.split()[0]) 
# This is just temporary for testing
nlines = 100000
print('Need to load %d lines' % nlines)

time_start = time.time()
total_start = time_start

# This gets our types and header for faster reads later
df = pd.read_csv(in_csv, nrows=2)
types = df.dtypes
columns = df.columns

# Ignore the header works, not using the csv thing because
for i in range(1, nlines, chunksize):
    print('reading lines: %d - %d' % (i-1, i+chunksize-1))
    df = pd.read_csv(
            in_csv,  
            header=None,
            names=columns,
            nrows=chunksize,
            skiprows=i,
            dtype=types
            )
    print(df.head(10))

print('overall time to read %d lines is %s' % (nlines, time.time() - total_start))

Need to load 100000 lines
reading lines: 0 - 10000
661
reading lines: 10000 - 20000
661
reading lines: 20000 - 30000
661
reading lines: 30000 - 40000
661
reading lines: 40000 - 50000
661
reading lines: 50000 - 60000
661
reading lines: 60000 - 70000
661
reading lines: 70000 - 80000
661
reading lines: 80000 - 90000
661
reading lines: 90000 - 100000
661
overall time to read 100000 lines is 20.8384599686


### Benchmarks
overall time to read 1000000 lines is 211.175414085